In [141]:
import pandas as pd
import sys
from tkinter import Tk, filedialog
import ast
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from tabulate import tabulate
import pprint 
import json
import scipy.stats as stats
import json
import pickle
import ipywidgets as widgets
from IPython.display import display
from nbconvert.preprocessors import ExecutePreprocessor
from nbformat import read, write
import nbformat
from functools import reduce
import statistics
from collections import OrderedDict

## Upload Database and Filter

In [147]:
# Load the DataFrame from the pkl file
file_path = 'K:/KayCei/Naturalistic.pkl'
with open(file_path, 'rb') as f:
    df = pickle.load(f)

# Identify columns with 'Array' in their names and parse JSON strings in those columns into Python objects
# Identify columns with 'Array' in their names and parse JSON strings in those columns into Python objects
df['Selected Behavior'] = df['Selected Behavior'].str.strip()
columns_to_parse=[]
for col in df.columns:
    if 'Array' in col:
        columns_to_parse.append(col)
#print(columns_to_parse)

for col in columns_to_parse:
    df[col] = df[col].apply(json.loads)

#def convert_to_tuples(item):
    #if isinstance(item, list):
        #return tuple(item)
    #else:
        #return item

# Apply the conversion function to the specified columns
#for col in columns_to_parse:
    #df[col] = df[col].apply(convert_to_tuples)

#Check if conversion was successful and print a statement accordingly
#conversion_successful = all(isinstance(item, tuple) for col in columns_to_parse for item in df[col])
#if conversion_successful:
    #print("JSON strings in columns with 'Array' in their names have been successfully converted to lists of tuples.")
#else:
    #print("Conversion of JSON strings to lists of tuples in columns with 'Array' in their names was unsuccessful.")

In [148]:
# Replace values in 'Selected_Behavior' column where 'Session_Type' is 'Sham Session'
df.loc[df['Session Type'] == 'Sham Session', 'Selected Behavior'] = 'No Error Value'
df.loc[df['Session Type'] == 'Sham Session', 'Fiber Connection'] = 'No Error Value'

In [149]:
original_df=df.copy()

original_df

In [150]:
iteration_list = {}

# Display available columns in groups of 3 with aligned numbering
num_columns = 3
columns = df.columns.tolist()
num_options = len(columns)

# Calculate the width of each column
column_width = 32

# Define the number of spaces for tab spacing
tab_space = 1

# Set the number of rows for displaying column options
num_rows = 13

# Initialize iteration number
iteration = 0
iteration_list[iteration]=("Selected Header: None","Column:None",df)

while columns:  # Continue filtering until all headers are filtered
    filtering_criteria = []
    for i in range(num_rows):
        for j in range(i, len(columns), num_rows):
            col_num = j + 1
            col_name = columns[j]
            # Adjust font size for smaller font
            if len(col_name) > column_width:
                print(f"\033[1;33m{col_num:<2}.\033[1;32m{col_name[:column_width]:<{column_width}}\033[0m", end=" " * tab_space)  # Yellow color for smaller font size
            else:
                print(f"\033[1;33m{col_num:<2}.{col_name:<{column_width}}\033[0m", end=" " * tab_space)
        print()
        
    if len(iteration_list)!=1:
        df=current_match
    else:
        pass
    
    # Prompt user to enter the number corresponding to the header
    selected_option = None
    while selected_option is None:
        user_input = input("Enter the number corresponding to the header or '0' to cancel filtering: ")
        try:
            selected_option = int(user_input)
            if selected_option == 0:
                print("Cell execution stopped.")
                sys.exit()  # Exit just the current cell
            elif selected_option > len(columns):
                raise ValueError
        except ValueError:
            print(f"Invalid input. Please enter a number between 1 and {len(columns)} or '0' to cancel filtering.")
            selected_option = None

    print((columns[selected_option-1]))
    selected_header = columns[selected_option - 1]
    
    # Append the selected header to filtering_criteria
    filtering_criteria.append(f"Selected Header: {selected_header}")

    # Remove the selected header from the list of available columns
    print(df[selected_header])    
    
    if selected_header in df.columns and all(isinstance(value, str) for value in df[selected_header]):
        avaliable_value=df[selected_header].unique()
        print(avaliable_value)
        filter_value = input(f"Enter the value to filter by {selected_header}: ")
        # Append the filter value to filtering_criteria
        filtering_criteria.append(f"Filter Value: {filter_value}")

    elif 'Array' in selected_header:
        if selected_header not in df.columns or df[selected_header].empty:
            print(f"The column '{selected_header}' is empty or does not exist.")
            
        available_elements = {
            0:"Behavior",
            1:"Stim Count",
            2: "Frames", 
            3: "Bout", 
            4: "Duration",
            5: "Start Time",
            6: "End Time",
        }
        print("Available elements:")
        for key, value in available_elements.items():
            print(f"{key}: {value}")

        # Prompt user to select elements
        element_positions_input = input(f"Enter the elements you want to filter by {selected_header} (comma-separated): ")
        print(df[selected_header])
        if ',' in element_positions_input:
            selected_positions = [int(pos.strip()) for pos in element_positions_input.split(',')]
            # Check if input is valid
            if all(pos in available_elements for pos in selected_positions):
                combined_filters = []
                for pos in selected_positions:
                    range_or_exact = input(f"Do you want an exact value or a range of values for {available_elements[pos]}?: ")
                    if range_or_exact.lower() == 'exact':
                        filter_value = input(f"Enter the exact value for {available_elements[pos]}: ")
                        y=int(filter_value)
                        filter_value=y
                        combined_filters.append(df[selected_header].apply(lambda tup_list: any((tup[pos] == filter_value for tup in tup_list))))
                        filtering_criteria.append(f"Element to filter: {available_elements.get(pos)}")
                        filtering_criteria.append(f"Exact Value for {available_elements[pos]}: {filter_value}")

                    elif range_or_exact.lower() == 'range':
                        range_value = input(f"Enter the range for {available_elements[pos]} (min,max): ")
                        min_val, max_val = map(float, range_value.split(','))
                        combined_filters.append(df[selected_header].apply(lambda tup_list: any((min_val <= tup[pos] <= max_val for tup in tup_list))))
                        filtering_criteria.append(f"Element to filter: {available_elements.get(pos)}")
                        filtering_criteria.append(f"Range for {available_elements[pos]}: {min_val} to {max_val}")

                    else:
                        print("Invalid input. Please enter 'exact' or 'range'.")

                # Apply combined filter
                array_df = df[reduce(lambda x, y: x & y, combined_filters)]

            else:
                print("Value is not an elemental position")

        else:
            selected_positions = int(element_positions_input)
            if selected_positions in available_elements:
                range_or_exact = input(f"Do you want an exact value or a range of values for {available_elements[selected_positions]}?: ")
                print(df[selected_header])
                if range_or_exact.lower() == 'exact':
                    print(df[selected_header])
                    filter_value= input(f"Enter the exact value for {available_elements[selected_positions]}: ")
                    filtering_criteria.append(f"Element to filter: {available_elements.get(selected_positions)}")
                    filtering_criteria.append(f"Exact Value for {available_elements.get(selected_positions)}: {filter_value}")
                    y=int(filter_value)
                    filter_value=y
                    print(df[selected_header])
                    array_df = df[df[selected_header].apply(lambda tup_list: any((tup[selected_positions] == filter_value for tup in tup_list)))]

                elif range_or_exact.lower() == 'range':
                    range_value = input(f"Enter the range for {available_elements[selected_positions]} (min,max): ")
                    min_val, max_val = map(float, range_value.split(','))
                    filtering_criteria.append(f"Element to filter: {available_elements.get(selected_positions)}")
                    filtering_criteria.append(f"Range for {available_elements.get(selected_positions)}: {min_val} to {max_val}")
                    array_df = df[df[selected_header].apply(lambda tup_list: any((min_val <= tup[selected_positions] <= max_val for tup in tup_list)))]

            else:
                print('An element position was not selected')
        
    if 'Array'not in selected_header:
        columns.remove(selected_header)
    
    if len(iteration_list)==1:
        if "Array" not in selected_header:                                          
            current_match=df[df[selected_header] == filter_value]
            iteration+=1
            iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match)
        elif "Selected Behavior" or 'Fiber Connection' in selected_header:
            current_match = df[(df[selected_header] == filter_value) | (df[selected_header] == "No Error Value")]
            iteration += 1
            iteration_list[iteration] = (filtering_criteria.copy(), columns.copy(), current_match)
        else:
            current_match=array_df
            iteration+=1
            iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match)                                       
    else:
        previous_match=current_match
        if'Array' in selected_header:
            current_match=array_df
            iteration+=1
            iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match) 
        elif "Selected Behavior" or 'Fiber Connection' in selected_header:
            current_match = df[(df[selected_header] == filter_value) | (df[selected_header] == "No Error Value")]
            iteration += 1
            iteration_list[iteration] = (filtering_criteria.copy(), columns.copy(), current_match)
        else:
            current_match=df[df[selected_header]==filter_value]
            iteration+=1
            iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match)                                        
    
    # Extracting the headers in the original order from the DataFrame
    original_headers = current_match.columns.tolist()



1 .Mouse                            14.Total Left Turn Bout Count       27.Face Groom Average Duration in S 
2 .Genotype                         15.Average Duration of Left Turn(se 28.All Other Groom Array            
3 .Date                             16.Total Left Turn Bouts in Stim Bl 29.Total All Other Groom Bout Count 
4 .Selected Behavior                17.Left Turn Average Duration in St 30.Average Duration of All Other Gr 
5 .Session Type                     18.Locomotion Array                 31.Total All Other Groom Bouts in S 
6 .Session Number                   19.Total Locomotion Bout Count      32.All Other Groom Average Duration 
7 .Fiber Connection                 20.Average Duration of Locomotion(s 33.All Other Array                  
8 .Right Turn Array                 21.Total Locomotion Bouts in Stim B 34.Total All Other Bout Count       
9 .Total Right Turn Bout Count      22.Locomotion Average Duration in S 35.Average Duration of All Other(se 
10.Average Duration

SystemExit: 

C:\Users\Yttri-Lab\anaconda3\envs\closed-loop\lib\site-packages\IPython\core\interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [151]:
array_df


,Mouse,Genotype,Date,Selected Behavior,Session Type,Session Number,Fiber Connection,Right Turn Array,Total Right Turn Bout Count,Average Duration of Right Turn(seconds),...,All Other Groom Array,Total All Other Groom Bout Count,Average Duration of All Other Groom(seconds),Total All Other Groom Bouts in Stim Block,All Other Groom Average Duration in Stim Block(seconds),All Other Array,Total All Other Bout Count,Average Duration of All Other(seconds),Total All Other Bouts in Stim Block,All Other Average Duration in Stim Block(seconds)
0,AD12,D1,021624,Right Turn,Stim Session,1,Bilateral,"[[1, 0, 3, 1, 1, 203, 204], [1, 0, 3, 2, 1, 21...",666,0.183,...,"[[4, 1, 6, 1, 2, 2663, 2665], [4, 0, 3, 2, 1, ...",253,0.182,134,0.16,"[[5, 0, 3, 1, 1, 0, 1], [5, 0, 3, 2, 1, 5, 6],...",3120,0.854,1371,0.9
1,AD12,D1,022024,Locomotion,Stim Session,1,Bilateral,"[[1, 0, 3, 1, 1, 13, 14], [1, 1, 9, 2, 3, 19, ...",789,0.199,...,"[[4, 0, 3, 1, 1, 2011, 2012], [4, 0, 3, 2, 1, ...",484,0.219,64,0.24,"[[5, 1, 12, 1, 4, 0, 4], [5, 3, 24, 2, 8, 5, 1...",3405,0.771,1517,0.78
2,AD12,D1,022324,Face Groom,Stim Session,1,Bilateral,"[[1, 1, 9, 1, 3, 0, 3], [1, 1, 6, 2, 2, 41, 43...",719,0.192,...,"[[4, 0, 3, 1, 1, 426, 427], [4, 0, 3, 2, 1, 45...",336,0.282,96,0.18,"[[5, 0, 3, 1, 1, 22, 23], [5, 1, 6, 2, 2, 25, ...",3148,0.9,1435,0.92
3,AD14,D1,022324,Right Turn,Stim Session,1,Bilateral,"[[1, 0, 3, 1, 1, 91, 92], [1, 1, 6, 2, 2, 198,...",361,0.172,...,"[[4, 1, 6, 1, 2, 3979, 3981], [4, 0, 3, 2, 1, ...",412,0.275,197,0.29,"[[5, 1, 12, 1, 4, 0, 4], [5, 4, 33, 2, 11, 9, ...",2953,0.925,1431,0.85
4,AD14,D1,022624,Locomotion,Stim Session,1,Bilateral,"[[1, 1, 12, 1, 4, 46, 50], [1, 0, 3, 2, 1, 62,...",249,0.143,...,"[[4, 0, 3, 1, 1, 97, 98], [4, 0, 3, 2, 1, 373,...",348,0.182,144,0.16,"[[5, 0, 3, 1, 1, 0, 1], [5, 1, 6, 2, 2, 12, 14...",2487,1.303,1072,1.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,P22,D1,071823,Right Turn,Stim Session,1,Contralateral,"[[0, 1, 6, 1, 2, 20, 22], [0, 2, 21, 2, 7, 41,...",1081,0.202,...,"[[4, 0, 3, 1, 1, 34, 35], [4, 1, 9, 2, 3, 117,...",732,0.304,301,0.25,"[[5, 2, 18, 1, 6, 0, 6], [5, 1, 6, 2, 2, 22, 2...",3341,0.793,1611,0.75
91,P22,D1,072023,Right Turn,Stim Session,2,Bilateral,"[[0, 0, 3, 1, 1, 94, 95], [0, 1, 6, 2, 2, 97, ...",1361,0.192,...,"[[4, 1, 9, 1, 3, 82, 85], [4, 0, 3, 2, 1, 93, ...",810,0.232,419,0.24,"[[5, 23, 207, 1, 69, 0, 69], [5, 1, 12, 2, 4, ...",3716,0.705,1803,0.67
92,P22,D1,072123,No Error Value,Sham Session,2,No Error Value,"[[0, 1, 6, 1, 2, 38, 40], [0, 1, 9, 2, 3, 54, ...",1122,0.205,...,"[[4, 1, 9, 1, 3, 114, 117], [4, 1, 9, 2, 3, 12...",678,0.237,305,0.22,"[[5, 2, 18, 1, 6, 3, 9], [5, 2, 15, 2, 5, 12, ...",3133,0.9,1471,0.88
93,P22,D1,072523,Locomotion,Stim Session,2,Bilateral,"[[0, 1, 9, 1, 3, 39, 42], [0, 0, 3, 2, 1, 45, ...",859,0.197,...,"[[4, 0, 3, 1, 1, 130, 131], [4, 0, 3, 2, 1, 15...",747,0.215,289,0.24,"[[5, 3, 30, 1, 10, 0, 10], [5, 2, 21, 2, 7, 11...",3456,0.736,1566,0.74


In [110]:
selected_header

'Right Turn Array'

In [132]:
if selected_positions in available_elements:
    print("true")
if range_or_exact.lower() == 'exact':
    print("true")
    print(filter_value)
    print(filtering_criteria)
    for col in df.columns:
        #print(col)
        if selected_header == col:
            #print("Its a match")
            y= df[selected_header]
            print (y)
            for tup_list in y:
                print(tup_list)
                for tup in tup_list:
                    if tup[selected_positions]==filter_value:
                        print(true)
                    else:
                        print(no)
    #array_df = df[df[selected_header].apply(lambda tup_list: any((tup[selected_positions] == filter_value for tup in tup_list)))]


true
true
5
[]
Series([], Name: Left Turn Array, dtype: object)


In [135]:
array_df

,Mouse,Genotype,Date,Selected Behavior,Session Type,Session Number,Fiber Connection,Right Turn Array,Total Right Turn Bout Count,Average Duration of Right Turn(seconds),...,All Other Groom Array,Total All Other Groom Bout Count,Average Duration of All Other Groom(seconds),Total All Other Groom Bouts in Stim Block,All Other Groom Average Duration in Stim Block(seconds),All Other Array,Total All Other Bout Count,Average Duration of All Other(seconds),Total All Other Bouts in Stim Block,All Other Average Duration in Stim Block(seconds)
0,AD12,D1,021624,Right Turn,Stim Session,1,Bilateral,"[[1, 0, 3, 1, 1, 203, 204], [1, 0, 3, 2, 1, 21...",666,0.183,...,"[[4, 1, 6, 1, 2, 2663, 2665], [4, 0, 3, 2, 1, ...",253,0.182,134,0.16,"[[5, 0, 3, 1, 1, 0, 1], [5, 0, 3, 2, 1, 5, 6],...",3120,0.854,1371,0.9
1,AD12,D1,022024,Locomotion,Stim Session,1,Bilateral,"[[1, 0, 3, 1, 1, 13, 14], [1, 1, 9, 2, 3, 19, ...",789,0.199,...,"[[4, 0, 3, 1, 1, 2011, 2012], [4, 0, 3, 2, 1, ...",484,0.219,64,0.24,"[[5, 1, 12, 1, 4, 0, 4], [5, 3, 24, 2, 8, 5, 1...",3405,0.771,1517,0.78
2,AD12,D1,022324,Face Groom,Stim Session,1,Bilateral,"[[1, 1, 9, 1, 3, 0, 3], [1, 1, 6, 2, 2, 41, 43...",719,0.192,...,"[[4, 0, 3, 1, 1, 426, 427], [4, 0, 3, 2, 1, 45...",336,0.282,96,0.18,"[[5, 0, 3, 1, 1, 22, 23], [5, 1, 6, 2, 2, 25, ...",3148,0.9,1435,0.92
3,AD14,D1,022324,Right Turn,Stim Session,1,Bilateral,"[[1, 0, 3, 1, 1, 91, 92], [1, 1, 6, 2, 2, 198,...",361,0.172,...,"[[4, 1, 6, 1, 2, 3979, 3981], [4, 0, 3, 2, 1, ...",412,0.275,197,0.29,"[[5, 1, 12, 1, 4, 0, 4], [5, 4, 33, 2, 11, 9, ...",2953,0.925,1431,0.85
4,AD14,D1,022624,Locomotion,Stim Session,1,Bilateral,"[[1, 1, 12, 1, 4, 46, 50], [1, 0, 3, 2, 1, 62,...",249,0.143,...,"[[4, 0, 3, 1, 1, 97, 98], [4, 0, 3, 2, 1, 373,...",348,0.182,144,0.16,"[[5, 0, 3, 1, 1, 0, 1], [5, 1, 6, 2, 2, 12, 14...",2487,1.303,1072,1.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,P22,D1,071823,Right Turn,Stim Session,1,Contralateral,"[[0, 1, 6, 1, 2, 20, 22], [0, 2, 21, 2, 7, 41,...",1081,0.202,...,"[[4, 0, 3, 1, 1, 34, 35], [4, 1, 9, 2, 3, 117,...",732,0.304,301,0.25,"[[5, 2, 18, 1, 6, 0, 6], [5, 1, 6, 2, 2, 22, 2...",3341,0.793,1611,0.75
91,P22,D1,072023,Right Turn,Stim Session,2,Bilateral,"[[0, 0, 3, 1, 1, 94, 95], [0, 1, 6, 2, 2, 97, ...",1361,0.192,...,"[[4, 1, 9, 1, 3, 82, 85], [4, 0, 3, 2, 1, 93, ...",810,0.232,419,0.24,"[[5, 23, 207, 1, 69, 0, 69], [5, 1, 12, 2, 4, ...",3716,0.705,1803,0.67
92,P22,D1,072123,No Error Value,Sham Session,2,No Error Value,"[[0, 1, 6, 1, 2, 38, 40], [0, 1, 9, 2, 3, 54, ...",1122,0.205,...,"[[4, 1, 9, 1, 3, 114, 117], [4, 1, 9, 2, 3, 12...",678,0.237,305,0.22,"[[5, 2, 18, 1, 6, 3, 9], [5, 2, 15, 2, 5, 12, ...",3133,0.9,1471,0.88
93,P22,D1,072523,Locomotion,Stim Session,2,Bilateral,"[[0, 1, 9, 1, 3, 39, 42], [0, 0, 3, 2, 1, 45, ...",859,0.197,...,"[[4, 0, 3, 1, 1, 130, 131], [4, 0, 3, 2, 1, 15...",747,0.215,289,0.24,"[[5, 3, 30, 1, 10, 0, 10], [5, 2, 21, 2, 7, 11...",3456,0.736,1566,0.74


In [134]:
if selected_header in df.columns:
    y = df[selected_header]
    
    # Check if the series is not empty
    if not y.empty:
        for tup_list in y:
            print(tup_list)
            for tup in tup_list:
                if tup[selected_positions] == filter_value:
                    print("true")
                else:
                    print("no")
    else:
        print("Selected header column is empty.")
else:
    print("Selected header column does not exist in the DataFrame.")

Selected header column is empty.


In [ ]:
for iteration, (filtering_criteria, columns, current_match) in iteration_list.items():
    print(iteration, filtering_criteria)

iteration_list.clear()
filtering_criteria.clear()

In [ ]:
len(current_match[current_match['Session Type'] == 'Stim Session'])

In [ ]:
while True:
    finalize_match = input("\nDo you want to finalize matching data? (yes/no): ")
    if finalize_match.lower() == 'yes':
        break  # Exit the loop if the user chooses to finalize matching data
    
    for iteration, (filtering_criteria, columns, current_match) in iteration_list.items():
        print(iteration, filtering_criteria)

    # Prompt the user to select an iteration to go back to
    selected_iteration = input("\nEnter the iteration number to go back to, or 'exit' to exit: ")
    if selected_iteration.lower() == 'exit':
        break  # Exit the loop if the user chooses to exit
    else:
        try:
            selected_iteration = int(selected_iteration)
            if selected_iteration in iteration_list:
                df=iteration_list[selected_iteration][2]
                current_match = iteration_list[selected_iteration][2]
                iteration_list = {k: v for k, v in iteration_list.items() if k <= selected_iteration}
                print('Run cell above as current match')
                break
        except ValueError:
            print("Invalid input. Please enter a valid iteration number or 'exit' to exit.")



In [ ]:
if finalize_match!='yes':
    # Define a list to store all selections and filtering criteria
    #iteration_list={}
    # Display available columns in groups of 3 with aligned numbering
    num_columns = 3
    columns = df.columns.tolist()
    num_options = len(columns)

    # Calculate the width of each column
    column_width = 32

    # Define the number of spaces for tab spacing
    tab_space = 1

    # Set the number of rows for displaying column options
    num_rows = 13

    while columns:  # Continue filtering until all headers are filtered
        for i in range(num_rows):
            for j in range(i, len(columns), num_rows):
                col_num = j + 1
                col_name = columns[j]
                # Adjust font size for smaller font
                if len(col_name) > column_width:
                    print(f"\033[1;33m{col_num:<2}.\033[1;32m{col_name[:column_width]:<{column_width}}\033[0m", end=" " * tab_space)  # Yellow color for smaller font size
                else:
                    print(f"\033[1;33m{col_num:<2}.{col_name:<{column_width}}\033[0m", end=" " * tab_space)
            print()

        if len(iteration_list)!=1:
            df=current_match
        else:
            pass
        filtering_criteria = []
        # Prompt user to enter the number corresponding to the header
        selected_option = None
        while selected_option is None:
            user_input = input("Enter the number corresponding to the header or '0' to cancel filtering: ")
            try:
                selected_option = int(user_input)
                if selected_option == 0:
                    print("Cell execution stopped.")
                    sys.exit()  # Exit just the current cell
                elif selected_option > len(columns):
                    raise ValueError
            except ValueError:
                print(f"Invalid input. Please enter a number between 1 and {len(columns)} or '0' to cancel filtering.")
                selected_option = None

        print((columns[selected_option-1]))
        selected_header = columns[selected_option - 1]

        # Append the selected header to filtering_criteria
        filtering_criteria.append(f"Selected Header: {selected_header}")

        # Remove the selected header from the list of available columns

        if selected_header in df.columns and all(isinstance(value, str) for value in df[selected_header]):
            avaliable_value=df[selected_header].unique()
            print(avaliable_value)
            filter_value = input(f"Enter the value to filter by {selected_header}: ")
            # Append the filter value to filtering_criteria
            filtering_criteria.append(f"Filter Value: {filter_value}")

        elif 'Array' in selected_header:
            available_elements = {
                0:"Behavior",
                1:"Stim Count",
                2: "Frames", 
                3: "Bout", 
                4: "Duration",
                5: "Start Time",
                6: "End Time",
            }
            print("Available elements:")
            for key, value in available_elements.items():
                print(f"{key}: {value}")

            # Prompt user to select elements
            element_positions_input = input(f"Enter the elements you want to filter by {selected_header} (comma-separated): ")
            print("Selected positions:", selected_positions)  # Add this line for debugging
            if ',' in element_positions_input:
                selected_positions = [int(pos.strip()) for pos in element_positions_input.split(',')]
                # Check if input is valid
                if all(pos in available_elements for pos in selected_positions):
                    combined_filters = []
                    for pos in selected_positions:
                        range_or_exact = input(f"Do you want an exact value or a range of values for {available_elements[pos]}?: ")
                        if range_or_exact.lower() == 'exact':
                            filter_value = input(f"Enter the exact value for {available_elements[pos]}: ")
                            # Add condition to the combined filter
                            combined_filters.append(df[selected_header].apply(lambda row: any((x[pos] == filter_value for x in tup)for tup in row)))
                            filtering_criteria.append(f"Element to filter: {available_elements.get(pos)}")
                            filtering_criteria.append(f"Exact Value for {available_elements[pos]}: {filter_value}")
                        elif range_or_exact.lower() == 'range':
                            range_value = input(f"Enter the range for {available_elements[pos]} (min,max): ")
                            min_val, max_val = map(float, range_value.split(','))
                            # Add condition to the combined filter
                            combined_filters.append(df[selected_header].apply(lambda row: any((min_val <= x[pos]<= max_val for x in tup)for tup in row)))
                            filtering_criteria.append(f"Element to filter: {available_elements.get(pos)}")
                            filtering_criteria.append(f"Range for {available_elements[pos]}: {min_val} to {max_val}")
                        else:
                            print("Invalid input. Please enter 'exact' or 'range'.")
                    # Apply combined filter
                    array_df = df[reduce(lambda x, y: x & y, combined_filters)]
                else:
                    print("Value is not an elemental position")
            else:
                selected_positions = int(element_positions_input)
                print(selected_positions)
                if selected_positions in available_elements:
                    range_or_exact = input(f"Do you want an exact value or a range of values for {available_elements[selected_positions]}?: ")
                    if range_or_exact.lower() == 'exact':
                        filter_value= input(f"Enter the exact value for {available_elements[selected_positions]}: ")
                        filtering_criteria.append(f"Element to filter: {available_elements.get(selected_positions)}")
                        filtering_criteria.append(f"Exact Value for {available_elements.get(selected_positions)}: {filter_value}")
                        array_df = df[df[selected_header].apply(lambda row: any((x[selected_positions] == filter_value for x in tup) for tup in row))]
                    elif range_or_exact== 'range':
                        range_value = input(f"Enter the range for {available_elements[selected_positions]} (min,max): ")
                        min_val, max_val = map(float, range_value.split(','))
                        filtering_criteria.append(f"Element to filter: {available_elements.get(selected_positions)}")
                        filtering_criteria.append(f"Range for {available_elements.get(selected_position)}: {min_val} to {max_val}")
                        array_df=df[df[selected_header].apply(lambda y: any((min_val <= x[selected_positions] <= max_val for x in tup)for tup in y))]
                else:
                    print('An element position was not selected')

        if 'Array'not in selected_header:
            columns.remove(selected_header)

        if len(iteration_list)==1:
            if "Array" in selected_header:                                          
                current_match=array_df
                iteration+=1
                iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match)
            elif "Selected Behavior" or 'Fiber Connection' in selected_header:
                current_match = df[(df[selected_header] == filter_value) | (df[selected_header] == "No Error Value")]
                iteration += 1
                iteration_list[iteration] = (filtering_criteria.copy(), columns.copy(), current_match)
            else:
                current_match=df[df[selected_header] == filter_value]
                iteration+=1
                iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match)                                       
        else:
            previous_match=current_match
            if'Array' in selected_header:
                current_match=array_df
                iteration+=1
                iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match) 
            elif "Selected Behavior" or 'Fiber Connection' in selected_header:
                current_match = df[(df[selected_header] == filter_value) | (df[selected_header] == "No Error Value")]
                iteration += 1
                iteration_list[iteration] = (filtering_criteria.copy(), columns.copy(), current_match)
            else:
                current_match=df[df[selected_header]==filter_value]
                iteration+=1
                iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match)                                       

        # Extracting the headers in the original order from the DataFrame
        original_headers = current_match.columns.tolist()
else:
    pass


In [ ]:
current_match

# Build Out Analysis:

# Q1: WAS THERE AN IN SESSION CHANGE ?

I would like to compare the change bout and duration for each session for each selected behavior for pre, stim, post. To do this I need to additional rows to the Dataframe. 
Step 1 ) Start w/Original DF 

In [ ]:
prepost_boutduration=original_df.copy()

In [ ]:
len(prepost_boutduration.columns)

In [ ]:
for col_name in prepost_boutduration.columns:
    if "Array" in col_name:
        action= col_name.split('Array')[0]
        preboutrowtocol=[]
        postboutrowtocol=[]
        predurationrowtocol=[]
        for element in prepost_boutduration[col_name]: #calls the row which is a  list of tuples
            prebout=0
            postbout=0
            preduration=[]
            postduration=[]
            for tup in element: # calls the tuples individually
                if tup[5]>=0 and tup[6]<= 3000:
                    prebout=+1
                    preduration.append(tup[4])
                if tup[5]>=21001 and tup[6]<=39000:
                    postbout=+1
                    postduration.append(tup[4])
            preboutrowtocol.append(prebout)
            postboutrowtocol.append(postbout)
            predurationaverage=statistics.mean(preduration)
            postdurationaverage=statistics.mean(postduration)
            predurationrowtocol.append(predurationaverage)
            postdurationrowtocol.append(postdurationaverage)
            
        prepost_boutduration[f'{action} Total Bouts Pre Stim']= preboutrowtocol
        prepost_boutduration[f'{action} Total Bouts Post Stim']=postboutrowtocol
        prepost_boutduration[f'{action} Average Duration Pre Stim']=predurationrowtocol
        prepost_boutduration[f'{action} Average Duration Post Stim']=postdurationrowtocol
        
prepost_boutduration[f'{action} Total Bouts Pre Stim']= preboutrowtocol
prepost_boutduration[f'{action} Total Bouts Post Stim']=postboutrowtocol
prepost_boutduration[f'{action} Average Duration Pre Stim']=predurationrowtocol
prepost_boutduration[f'{action} Average Duration Post Stim']=postdurationrowtocol

        

## Total Pre, Stim, Post of All Behaviors

### Step 1: Bout

### Step 2

### Step 3

## Step 1: Duration

In [ ]:
duration_pre_stim_post={}

count_pre=[]
count_stim=[]
count_post=[]
pre_rowdur=[]
stim_rowdur=[]
post_rowdur=[]

for col in current_match.columns:
    if "Array" in col:
        dur_col = col.split("Array")[0].strip()
        for row in current_match[col]:
            for tup in row:
                if 0<=tup[5] and tup[6]<= 3000:
                    count_pre.append(tup[4])
                if tup[5]>=3001 and tup[6]<=21000:
                    count_stim.append(tup[4])
                if tup[5]>=21001 and tup[6]<=39001:
                    count_post.append(tup[4])
                    
            #After each row add up the sum of the values 
            pre_dur=sum(count_pre)
            stim_dur=sum(count_stim)
            post_dur=sum(count_post)
            
            #Append row totals to the row list 
            pre_rowdur.append(pre_dur)
            stim_rowdur.append(stim_dur)
            post_rowdur.append(post_dur)
            
            #Clear row values
            count_pre.clear()
            count_stim.clear()
            count_post.clear()
            
        #average the values of all the rows of pre stim post and make the values seconds 
        x= int(statistics.mean(pre_rowdur)/10)
        y= int(statistics.mean(stim_rowdur)/10)
        z= int(statistics.mean(post_rowdur)/10)
           
        #append to the suration dictionary Key(Behavior): Value (pre, stim, post total)
        duration_pre_stim_post[dur_col]=(x, y,z)
           
        #clear the columns averages for next column 
        pre_rowdur.clear()
        stim_rowdur.clear()
        post_rowdur.clear()


In [ ]:
index_labels=['Pre','Stim','Post']
totalduration=pd.DataFrame(duration_pre_stim_post,index_labels)
totalduration


### Duration Plot of Average Pre Post Stim During Selected Action 

In [ ]:
# Simulating the condition for selecting a behavior
selected_behavior = current_match['Selected Behavior'].iloc[0]
print(selected_behavior)
selected_genotype = current_match['Genotype'].iloc[0]

# Reorder the rows such that the row matching the condition is the first row
if selected_behavior in totalduration.columns:
    selected_col=totalduration.pop(selected_behavior)
    totalduration.insert(0,selected_behavior,selected_col)

# Transpose the DataFrame to have columns as x values
totalduration_transposed = totalduration.transpose()

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))

# Plot the transposed DataFrame
totalduration_transposed.plot(kind='bar', ax=ax, width=0.5,color=['#d0e1f9', '#283655', '#1e1f26'])

ax.set_ylabel('Time (s)')

# Set the title based on selected_behavior
if selected_behavior == 'Right Turn':
    ax.set_title(f"{selected_genotype} {selected_behavior} {current_match['Fiber Connection'].iloc[0]} Average Duration ({current_match['Session Type'].iloc[0]})")
else:
    ax.set_title(f"{selected_genotype} {selected_behavior} Average Duration ({current_match['Session Type'].iloc[0]})")

# Set the x-axis label
ax.set_xlabel('Behavior')

plt.legend(title='Period', loc='upper left')

plt.tight_layout()

# Set the current figure as the active figure
plt.gcf()

# Specify the folder path
folder_path = 'K:/KayCei/Naturalistic Closed Loop/Totals/Duration Plots'

# Save the plot as a PNG image with increased DPI
title_str = ax.get_title()
filename = ''.join(e for e in title_str if e.isalnum())  # Remove special characters and spaces
plt.savefig(os.path.join(folder_path, f'{filename}.png'), dpi=300)  # Save as PNG file with increased DPI in the specif


plt.show()


In [ ]:
plt.close()

In [ ]:
# Check if sham_stim_duration is already initialized
if 'sham_stim_duration' not in locals():
    sham_stim_duration = []

# Define the new tuple
new_tuple = (title_str, totalduration_transposed)

# Check if the first element of the new tuple already exists in sham_stim_duration
existing_titles = [tup[0] for tup in sham_stim_duration]
if title_str not in existing_titles:
    sham_stim_duration.append(new_tuple)


In [ ]:
d2stimsham={}
d1stimsham={}

for tup in sham_stim_duration:
    if "D1" in tup[0]:
        d1stimsham[tup[0]]=tup[1]
    else:
        if "D2" in tup[0]:
            d2stimsham[tup[0]]=tup[1]



In [ ]:
# Define a function to extract "sham" DataFrame from a given dictionary
def extract_sham_data(data_dict):
    sham_title, sham_df = None, None
    for key, value in data_dict.items():
        if "Sham Session" in key:
            sham_title, sham_df = key, value
            data_dict.pop(key)
            break
    return sham_title, sham_df

# Extract "sham" DataFrame from d1stimsham
sham_title_d1, sham_df_d1 = extract_sham_data(d1stimsham)
if sham_df_d1 is None:
    print("No 'sham' data found in d1stimsham.")
else:
    print(f"Extracted 'sham' data from d1stimsham with title '{sham_title_d1}'.")

# Extract "sham" DataFrame from d2stimsham
sham_title_d2, sham_df_d2 = extract_sham_data(d2stimsham)
if sham_df_d2 is None:
    print("No 'sham' data found in d2stimsham.")
else:
    print(f"Extracted 'sham' data from d2stimsham with title '{sham_title_d2}'.")

# Iterate over each DataFrame in d1stimsham and compare it to the transposed "sham" DataFrame
for title, df in d1stimsham.items():
    if df is sham_df_d1:
        continue  # Skip if the DataFrame is the "sham" DataFrame
    
    # Plot the transposed DataFrames
    fig, ax = plt.subplots(figsize=(10, 6))
    
    df.plot(kind='bar', ax=ax, width=0.5, color=['#d0e1f9', '#283655', '#1e1f26'])
    sham_df_d1.plot(kind='bar', ax=ax, width=0.5, color=['#FF5733', '#FF5733', '#FF5733'], alpha=0.7)

    ax.set_ylabel('Time (s)')
    ax.set_xlabel('Behavior')
    ax.set_title(f"Comparison of {title} vs Sham (d1stimsham)")
    
    plt.legend(["Pre", 'Stim', 'Post', 'Sham'], title='Period', loc='upper left')
    
    plt.tight_layout()
    
    plt.show()

# Iterate over each DataFrame in d2stimsham and compare it to the transposed "sham" DataFrame
for title, df in d2stimsham.items():
    if df is sham_df_d2:
        continue  # Skip if the DataFrame is the "sham" DataFrame
    
    # Plot the transposed DataFrames
    fig, ax = plt.subplots(figsize=(10, 6))
    
    df.plot(kind='bar', ax=ax, width=0.5, color=['#d0e1f9', '#283655', '#1e1f26'])
    sham_df_d2.plot(kind='bar', ax=ax, width=0.5, color=['#FF5733', '#FF5733', '#FF5733'], alpha=0.7)

    ax.set_ylabel('Time (s)')
    ax.set_xlabel('Behavior')
    ax.set_title(f"Comparison of {title} vs Sham (d2stimsham)")
    
    plt.legend(["Pre", 'Stim', 'Post', 'Sham'], title='Period', loc='upper left')
    
    plt.tight_layout()
    
    plt.show()

In [ ]:
# Extract the "sham" DataFrame
sham_title, sham_df = None, None
for key, value in d2stimsham.items():
    if "Sham Session" in key:
        sham_title, sham_df = key, value
        d2stimsham.pop(key)
        break

if sham_df is None:
    print("No 'sham' data found.")
else:
    print(f"Extracted 'sham' data with title '{sham_title}'.")

# Iterate over each DataFrame and compare it to the transposed "sham" DataFrame
for title, df in d2stimsham.items():
    if df is sham_df:
        continue  # Skip if the DataFrame is the "sham" DataFrame
    
    # Plot the transposed DataFrames
    fig, ax = plt.subplots(figsize=(10, 6))
    
    df.plot(kind='bar', ax=ax, width=0.5, color=['#d0e1f9', '#283655', '#1e1f26'])
    sham_df.plot(kind='bar', ax=ax, width=0.5, color=['#FF5733', '#FF5733', '#FF5733'], alpha=0.7)

    ax.set_ylabel('Time (s)')
    ax.set_xlabel('Behavior')
    ax.set_title(f"Comparison of {title} vs Sham")
    
    plt.legend(["Pre", 'Stim', 'Post', 'Sham'], title='Period', loc='upper left')
    
    plt.tight_layout()
    
    plt.show()

In [ ]:
current_match.columns

In [ ]:

# Step 1: Statistical Methods
# Calculate summary statistics
for col in current_match.columns:
    if "Average Duration in Stim Block(seconds)" in col:
        col_name=col
        duration_data = current_match[col]
        summary_stats = duration_data.describe()

        # Calculate IQR (Interquartile Range)
        Q1 = duration_data.quantile(0.25)
        Q3 = duration_data.quantile(0.75)
        IQR = Q3 - Q1

        # Identify outliers using IQR method
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = duration_data[(duration_data < lower_bound) | (duration_data > upper_bound)]

        # Step 2: Visualization Techniques
        # Create a box plot to visualize the distribution and identify outliers
        plt.figure(figsize=(8, 6))
        plt.boxplot(duration_data, vert=False)
        plt.title(f'Box Plot of {col_name}')
        plt.xlabel(col_name)
        plt.grid(True)
        plt.show()

        # Create a histogram to visualize the frequency distribution
        plt.figure(figsize=(8, 6))
        plt.hist(duration_data, bins=20, color='skyblue', edgecolor='black')
        plt.title(f'Histogram of {col_name}')
        plt.xlabel(col_name)
        plt.ylabel('Frequency')
        plt.grid(True)
        plt.show()

        # Print summary statistics and outliers
        print("Summary Statistics:")
        print(summary_stats)
        print("\nOutliers:")
        print(outliers)

## Stim vs Sham (Repeated ANOVA)

In [ ]:
while True:
    finalize_match = input("\nDo you want to finalize matching data? (yes/no): ")
    if finalize_match.lower() == 'yes':
        break  # Exit the loop if the user chooses to finalize matching data
    
    for iteration, (filtering_criteria, columns, current_match) in iteration_list.items():
        print(iteration, filtering_criteria)

    # Prompt the user to select an iteration to go back to
    selected_iteration = input("\nEnter the iteration number to go back to, or 'exit' to exit: ")
    if selected_iteration.lower() == 'exit':
        break  # Exit the loop if the user chooses to exit
    else:
        try:
            selected_iteration = int(selected_iteration)
            if selected_iteration in iteration_list:
                df=iteration_list[selected_iteration][2]
                current_match = iteration_list[selected_iteration][2]
                iteration_list = {k: v for k, v in iteration_list.items() if k <= selected_iteration}
                print('Run cell above as current match')
                break
        except ValueError:
            print("Invalid input. Please enter a valid iteration number or 'exit' to exit.")



In [ ]:
if finalize_match!='yes':
    # Define a list to store all selections and filtering criteria
    #iteration_list={}
    # Display available columns in groups of 3 with aligned numbering
    num_columns = 3
    columns = df.columns.tolist()
    num_options = len(columns)

    # Calculate the width of each column
    column_width = 32

    # Define the number of spaces for tab spacing
    tab_space = 1

    # Set the number of rows for displaying column options
    num_rows = 13

    while columns:  # Continue filtering until all headers are filtered
        for i in range(num_rows):
            for j in range(i, len(columns), num_rows):
                col_num = j + 1
                col_name = columns[j]
                # Adjust font size for smaller font
                if len(col_name) > column_width:
                    print(f"\033[1;33m{col_num:<2}.\033[1;32m{col_name[:column_width]:<{column_width}}\033[0m", end=" " * tab_space)  # Yellow color for smaller font size
                else:
                    print(f"\033[1;33m{col_num:<2}.{col_name:<{column_width}}\033[0m", end=" " * tab_space)
            print()

        if len(iteration_list)!=1:
            df=current_match
        else:
            pass
        filtering_criteria = []
        # Prompt user to enter the number corresponding to the header
        selected_option = None
        while selected_option is None:
            user_input = input("Enter the number corresponding to the header or '0' to cancel filtering: ")
            try:
                selected_option = int(user_input)
                if selected_option == 0:
                    print("Cell execution stopped.")
                    sys.exit()  # Exit just the current cell
                elif selected_option > len(columns):
                    raise ValueError
            except ValueError:
                print(f"Invalid input. Please enter a number between 1 and {len(columns)} or '0' to cancel filtering.")
                selected_option = None

        print((columns[selected_option-1]))
        selected_header = columns[selected_option - 1]

        # Append the selected header to filtering_criteria
        filtering_criteria.append(f"Selected Header: {selected_header}")

        # Remove the selected header from the list of available columns

        if selected_header in df.columns and all(isinstance(value, str) for value in df[selected_header]):
            avaliable_value=df[selected_header].unique()
            print(avaliable_value)
            filter_value = input(f"Enter the value to filter by {selected_header}: ")
            # Append the filter value to filtering_criteria
            filtering_criteria.append(f"Filter Value: {filter_value}")

        elif 'Array' in selected_header:
            available_elements = {
                0:"Behavior",
                1:"Stim Count",
                2: "Frames", 
                3: "Bout", 
                4: "Duration",
                5: "Start Time",
                6: "End Time",
            }
            print("Available elements:")
            for key, value in available_elements.items():
                print(f"{key}: {value}")

            # Prompt user to select elements
            element_positions_input = input(f"Enter the elements you want to filter by {selected_header} (comma-separated): ")
            print("Selected positions:", selected_positions)  # Add this line for debugging
            if ',' in element_positions_input:
                selected_positions = [int(pos.strip()) for pos in element_positions_input.split(',')]
                # Check if input is valid
                if all(pos in available_elements for pos in selected_positions):
                    combined_filters = []
                    for pos in selected_positions:
                        range_or_exact = input(f"Do you want an exact value or a range of values for {available_elements[pos]}?: ")
                        if range_or_exact.lower() == 'exact':
                            filter_value = input(f"Enter the exact value for {available_elements[pos]}: ")
                            # Add condition to the combined filter
                            combined_filters.append(df[selected_header].apply(lambda row: any((x[pos] == filter_value for x in tup)for tup in row)))
                            filtering_criteria.append(f"Element to filter: {available_elements.get(pos)}")
                            filtering_criteria.append(f"Exact Value for {available_elements[pos]}: {filter_value}")
                        elif range_or_exact.lower() == 'range':
                            range_value = input(f"Enter the range for {available_elements[pos]} (min,max): ")
                            min_val, max_val = map(float, range_value.split(','))
                            # Add condition to the combined filter
                            combined_filters.append(df[selected_header].apply(lambda row: any((min_val <= x[pos]<= max_val for x in tup)for tup in row)))
                            filtering_criteria.append(f"Element to filter: {available_elements.get(pos)}")
                            filtering_criteria.append(f"Range for {available_elements[pos]}: {min_val} to {max_val}")
                        else:
                            print("Invalid input. Please enter 'exact' or 'range'.")
                    # Apply combined filter
                    array_df = df[reduce(lambda x, y: x & y, combined_filters)]
                else:
                    print("Value is not an elemental position")
            else:
                selected_positions = int(element_positions_input)
                print(selected_positions)
                if selected_positions in available_elements:
                    range_or_exact = input(f"Do you want an exact value or a range of values for {available_elements[selected_positions]}?: ")
                    if range_or_exact.lower() == 'exact':
                        filter_value= input(f"Enter the exact value for {available_elements[selected_positions]}: ")
                        filtering_criteria.append(f"Element to filter: {available_elements.get(selected_positions)}")
                        filtering_criteria.append(f"Exact Value for {available_elements.get(selected_positions)}: {filter_value}")
                        array_df = df[df[selected_header].apply(lambda row: any((x[selected_positions] == filter_value for x in tup) for tup in row))]
                    elif range_or_exact== 'range':
                        range_value = input(f"Enter the range for {available_elements[selected_positions]} (min,max): ")
                        min_val, max_val = map(float, range_value.split(','))
                        filtering_criteria.append(f"Element to filter: {available_elements.get(selected_positions)}")
                        filtering_criteria.append(f"Range for {available_elements.get(pos)}: {min_val} to {max_val}")
                        array_df=df[df[selected_header].apply(lambda y: any((min_val <= x[selected_positions] <= max_val for x in tup)for tup in y))]
                else:
                    print('An element position was not selected')

        if 'Array'not in selected_header:
            columns.remove(selected_header)

        if len(iteration_list)==1:
            if "Array" not in selected_header:                                          
                current_match=df[df[selected_header] == filter_value]
                iteration+=1
                iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match)
            elif "Selected Behavior" or 'Fiber Connection' in selected_header:
                current_match = df[(df[selected_header] == filter_value) | (df[selected_header] == "No Error Value")]
                iteration += 1
                iteration_list[iteration] = (filtering_criteria.copy(), columns.copy(), current_match)
            else:
                current_match=array_df
                iteration+=1
                iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match)                                       
        else:
            previous_match=current_match
            if'Array' in selected_header:
                current_match=array_df
                iteration+=1
                iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match) 
            elif "Selected Behavior" or 'Fiber Connection' in selected_header:
                current_match = df[(df[selected_header] == filter_value) | (df[selected_header] == "No Error Value")]
                iteration += 1
                iteration_list[iteration] = (filtering_criteria.copy(), columns.copy(), current_match)
            else:
                current_match=df[df[selected_header]==filter_value]
                iteration+=1
                iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match)                                       

        # Extracting the headers in the original order from the DataFrame
        original_headers = current_match.columns.tolist()
else:
    pass


In [ ]:
anovadf=current_match.copy()

In [ ]:
count_pre = []
count_stim = []
count_post = []
num_array_columns = sum("Array" in col for col in anovadf.columns)
current_iteration = 0

for col in anovadf.columns:
    if "Array" in col:
        current_iteration += 1
        bout_col_1 = col.split("Array")[0].strip()
        bout_col = '_'.join(bout_col_1.split())
        print(bout_col)
        for row in anovadf[col]:
            pre_count=0
            stim_count=0
            post_count=0
            for tup in row:
                if 0<=tup[5] and tup[6]<= 3000:
                    pre_count+=1
                if tup[5]>=3001 and tup[6]<=21000:
                    stim_count+=1
                if tup[5]>=21001 and tup[6]<=39001:
                    post_count+=1
            count_pre.append(pre_count)
            count_stim.append(stim_count)
            count_post.append(post_count)

        # Add new columns to anovadf before moving to the next array column
        anovadf[f'{bout_col}_Pre_Bout'] = count_pre
        anovadf[f'{bout_col}_Stim_Bout_Count'] = count_stim
        anovadf[f'{bout_col}_Post_Bout'] = count_post

        
        # Clear the count lists if it's not the last iteration
        if current_iteration != num_array_columns:
            count_pre.clear()
            count_stim.clear()
            count_post.clear()

# Add remaining counts to the DataFrame if needed
anovadf[f'{bout_col}_Pre_Bout'] = count_pre
anovadf[f'{bout_col}_Stim_Bout_Count'] = count_stim
anovadf[f'{bout_col}_Post_Bout'] = count_post

In [ ]:
print(anovadf.columns)

In [ ]:
anovadf.rename(columns={'Session Type': 'Session_Type'}, inplace=True)

In [ ]:
anovadf

In [ ]:
len(anovadf[anovadf['Session_Type'] == 'Stim Session'])

In [ ]:
# Filter columns with '_Stim_Bout_Count' in header
filtered_columns = [col for col in anovadf.columns if '_Stim_Bout_Count' in col]

# Filter columns with 'Bouts in Stim Block' in header
filtered_columns_1 = [col for col in anovadf.columns if 'Bouts in Stim Block' in col]

# List to store column pairs with matching values
matching_columns = []

# Iterate over columns pairwise
for col1 in filtered_columns:
    for col2 in filtered_columns_1:
        # Check if values in columns col1 and col2 match at the same index
        if (anovadf[col1] == anovadf[col2]).all():
            matching_columns.append((col1, col2))

print("Columns with matching values at the same index:")
for pair in matching_columns:
    print(pair)

In [ ]:
import statsmodels.api as sm
from statsmodels.multivariate.manova import MANOVA

In [ ]:
# Define the dependent variables
dependent_vars = ['Right_Turn_Pre_Bout', 'Right_Turn_Stim_Bout_Count',
       'Right_Turn_Post_Bout', 'Left_Turn_Pre_Bout',
       'Left_Turn_Stim_Bout_Count', 'Left_Turn_Post_Bout',
       'Locomotion_Pre_Bout', 'Locomotion_Stim_Bout_Count',
       'Locomotion_Post_Bout', 'Face_Groom_Pre_Bout',
       'Face_Groom_Stim_Bout_Count', 'Face_Groom_Post_Bout',
       'All_Other_Groom_Pre_Bout', 'All_Other_Groom_Stim_Bout_Count',
       'All_Other_Groom_Post_Bout', 'All_Other_Pre_Bout',
       'All_Other_Stim_Bout_Count', 'All_Other_Post_Bout']

# Create a copy of the original dataframe
filtered_data = anovadf.copy()

# Filter out zero values for each dependent variable separately
for var in dependent_vars:
    filtered_data = filtered_data[filtered_data[var] != 0]

# Perform MANOVA on filtered data
manova_model = MANOVA.from_formula(" + ".join(dependent_vars) + " ~ Session_Type", data=filtered_data)
manova_results = manova_model.mv_test()

# Define the title for the Multivariate Linear Model section
stim_session_rows = filtered_data[filtered_data['Session_Type'] == 'Stim Session']
selected_behavior = stim_session_rows['Selected Behavior'].iloc[0]
fiber_connection = stim_session_rows['Fiber Connection'].iloc[0]

title = f"{selected_behavior} {fiber_connection} Multivariate Linear Model (excluding zeros for each dependent variable)"

# Print results with modified title
print(title.center(60, '='))
print(manova_results.summary())


In [ ]:
# Define the dependent variables
dependent_vars = ['Right_Turn_Pre_Bout', 'Right_Turn_Stim_Bout_Count',
       'Right_Turn_Post_Bout', 'Left_Turn_Pre_Bout',
       'Left_Turn_Stim_Bout_Count', 'Left_Turn_Post_Bout',
       'Locomotion_Pre_Bout', 'Locomotion_Stim_Bout_Count',
       'Locomotion_Post_Bout', 'Face_Groom_Pre_Bout',
       'Face_Groom_Stim_Bout_Count', 'Face_Groom_Post_Bout',
       'All_Other_Groom_Pre_Bout', 'All_Other_Groom_Stim_Bout_Count',
       'All_Other_Groom_Post_Bout', 'All_Other_Pre_Bout',
       'All_Other_Stim_Bout_Count', 'All_Other_Post_Bout']

# Extract variables containing the substring "Stim_Bout_Count"
Bout_stim_vars = [var for var in dependent_vars if 'Stim_Bout_Count' in var]

# Create a copy of the original dataframe
filtered_data = anovadf.copy()

# Filter out zero values for each dependent variable separately
for var in Bout_stim_vars:
    filtered_data = filtered_data[filtered_data[var] != 0]

# Perform MANOVA on filtered data
manova_model = MANOVA.from_formula(" + ".join(Bout_stim_vars) + " ~ Session_Type", data=filtered_data)
manova_results = manova_model.mv_test()

# Define the title for the Multivariate Linear Model section
stim_session_rows = filtered_data[filtered_data['Session_Type'] == 'Stim Session']
selected_behavior = stim_session_rows['Selected Behavior'].iloc[0]
fiber_connection = stim_session_rows['Fiber Connection'].iloc[0]

title = f"{selected_behavior} {fiber_connection} Multivariate Linear Model (excluding zeros for each dependent variable)"

# Print results with modified title
print(title.center(60, '='))
print(manova_results.summary())

In [ ]:


# Define the file path
file_path = "/Users/kaycei/K on Server (NoMachine)/KayCei/Naturalistic Closed Loop/Totals/Multivariance with manova"

# Save MANOVA results to a file with the title as the filename
filename = os.path.join(file_path, f"{title}.csv")
manova_results_summary = manova_results.summary().as_text()
with open(filename, 'w') as f:
    f.write(manova_results_summary)


In [ ]:
anovadf

In [ ]:
import pandas as pd
from scipy.stats import ttest_rel

# Define the group names and corresponding columns from anovadf
group_columns = {
    'Right': ['Right_Turn_Pre_Bout', 'Right_Turn_Stim_Bout_Count', 'Right_Turn_Post_Bout'],
    'Left': ['Left_Turn_Pre_Bout', 'Left_Turn_Stim_Bout_Count', 'Left_Turn_Post_Bout'],
    'Locomotion': ['Locomotion_Pre_Bout', 'Locomotion_Stim_Bout_Count', 'Locomotion_Post_Bout'],
    'Face Groom':['Face_Groom_Pre_Bout','Face_Groom_Stim_Bout_Count', 'Face_Groom_Post_Bout',],
    'All Other Groom':['All_Other_Groom_Pre_Bout', 'All_Other_Groom_Stim_Bout_Count','All_Other_Groom_Post_Bout'],
    'All Other':['All_Other_Pre_Bout','All_Other_Stim_Bout_Count', 'All_Other_Post_Bout'],
}

# Initialize dictionaries to store results
group_comparison_results = {}
pairwise_comparison_results = {}

# Step 1: Subgroup Analysis and Pairwise Comparisons
for group, columns in group_columns.items():
    # Select columns related to the current group from anovadf
    group_data = anovadf[columns]

    # Separate data into "Stim" and "Sham" sessions
    stim_data_1 = group_data[anovadf['Session_Type'] == 'Stim Session']
    sham_data_1 = group_data[anovadf['Session_Type'] == 'Sham Session']

    # Ensure that the number of observations in "Stim" and "Sham" sessions are equal
    min_obs = min(len(stim_data_1), len(sham_data_1))
    if min_obs == 0:
        continue  # Skip this group if one of the sessions has no data
    stim_data = stim_data.sample(min_obs, replace=True, random_state=42)
    sham_data = sham_data.sample(min_obs, replace=True, random_state=42)

    # Perform paired t-test for "Stim" vs. "Sham" within the current group
    t_stat, p_value = ttest_rel(stim_data.values.flatten(), sham_data.values.flatten())
    group_comparison_results[group] = {'t_stat': t_stat, 'p_value': p_value}

    # Perform pairwise comparisons across groups
    other_groups = [other_group for other_group in group_columns.keys() if other_group != group]
    pairwise_comparison_results[group] = {}
    for other_group in other_groups:
        other_group_data = anovadf[group_columns[other_group]]

        # Perform paired t-test between the current group and other groups
        t_stat, p_value = ttest_rel(group_data.values.flatten(), other_group_data.values.flatten())
        pairwise_comparison_results[group][other_group] = {'t_stat': t_stat, 'p_value': p_value}

In [ ]:
# Print subgroup analysis results
print("Subgroup Analysis Results:")
for group, result in group_comparison_results.items():
    print(f"Group: {group}")
    print(f"T-statistic: {result['t_stat']}")
    print(f"P-value: {result['p_value']}")
    print()

# Print pairwise comparison results
print("Pairwise Comparison Results:")
for group, comparisons in pairwise_comparison_results.items():
    print(f"Group: {group}")
    for other_group, result in comparisons.items():
        print(f"Comparison with {other_group}:")
        print(f"T-statistic: {result['t_stat']}")
        print(f"P-value: {result['p_value']}")
        print()

In [ ]:
# Define the group names and corresponding columns
group_columns = {
    'Right': ['Right_Turn_Stim_Bout_Count'],
    'Left': ['Left_Turn_Stim_Bout_Count'],
    'Locomotion': ['Locomotion_Stim_Bout_Count'],
    'Face Groom': ['Face_Groom_Stim_Bout_Count'],
    'All Other Groom': ['All_Other_Groom_Stim_Bout_Count'],
    'All Other': ['All_Other_Stim_Bout_Count'],
}

# Initialize dictionaries to store results
group_comparison_results = {}
pairwise_comparison_results = {}

# Step 1: Subgroup Analysis and Pairwise Comparisons
for group, columns in group_columns.items():
    # Select columns related to the current group from anovadf
    group_data = anovadf[columns]

    # Separate data into "Stim" and "Sham" sessions
    stim_data = group_data[anovadf['Session_Type'] == 'Stim Session']
    sham_data = group_data[anovadf['Session_Type'] == 'Sham Session']

    # Ensure that the number of observations in "Stim" and "Sham" sessions are equal
    min_obs = min(len(stim_data), len(sham_data))
    stim_data = stim_data.sample(min_obs, random_state=42)
    sham_data = sham_data.sample(min_obs, random_state=42)

    # Perform paired t-test for "Stim" vs. "Sham" within the current group
    t_stat, p_value = ttest_rel(stim_data.values.flatten(), sham_data.values.flatten())
    group_comparison_results[group] = {'t_stat': t_stat, 'p_value': p_value}

    # Perform pairwise comparisons across groups
    other_groups = [other_group for other_group in group_columns.keys() if other_group != group]
    pairwise_comparison_results[group] = {}
    for other_group in other_groups:
        other_group_data = anovadf[group_columns[other_group]]

        # Perform paired t-test between the current group and other groups
        t_stat, p_value = ttest_rel(group_data.values.flatten(), other_group_data.values.flatten())
        pairwise_comparison_results[group][other_group] = {'t_stat': t_stat, 'p_value': p_value}

In [ ]:
# Print subgroup analysis results
print("Subgroup Analysis Results:")
for group, result in group_comparison_results.items():
    print(f"Group: {group}")
    print(f"T-statistic: {result['t_stat']}")
    print(f"P-value: {result['p_value']}")
    print()

# Print pairwise comparison results
print("Pairwise Comparison Results:")
for group, comparisons in pairwise_comparison_results.items():
    print(f"Group: {group}")
    for other_group, result in comparisons.items():
        print(f"Comparison with {other_group}:")
        print(f"T-statistic: {result['t_stat']}")
        print(f"P-value: {result['p_value']}")
        print()

In [ ]:
len(sham_data_1)

In [ ]:
len(stim_data_1)

In [ ]:
stim_data

In [ ]:
print(min_obs)